In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import concurrent
import pickle

import matplotlib
from mpl_toolkits import mplot3d
font = {'size'   : 16}
matplotlib.rc('font', **font)

In [2]:
file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\magnet_calibration_test_current_hpcalib.h5"
print(os.path.isfile(file_path))
out_folder = 'test_results1'
try:
    os.stat(out_folder)
except:
    os.mkdir(out_folder) 
poles = [0, 1, 2]
poles_dict = {0:'A', 1:'B', 2:'C'}

True


In [53]:
def filter_data(data, bin_step=100):
    # to speed up, filter the high frequency stuff
    t = data[:, 0].copy()
    n = int(len(t)/bin_step)
    data_bin = np.zeros((n, data.shape[1]))
    for i in range(data.shape[1]):
        data_bin[:, i] = binned_statistic(t, data[:, i], bins=int(len(t)/bin_step), statistic='mean').statistic
    return data_bin
# import rl coefficients
with open('rl_coeff.p', 'rb') as f:
    rl_coeff = pickle.load(f)

### Test the sin signal generation

In [63]:
plt.plot(hx_data[:, 1:])

In [ ]:
%matplotlib qt
# import the data
# pole = 0

directions_dict = ['X', 'Y', 'Z']
errcolor = 'tab:green'
with h5py.File(file_path, 'r') as f:
    for pole in poles:
        print(pole)
        pole_grp = f['frequency_response/pole{}'.format(pole)]
        for i, loop_grp in enumerate(pole_grp.values()):
#             if i != 15:
#                 continue
            # get the desired signal
            damp = loop_grp.attrs['desired_amplitude']
            dfreq = loop_grp.attrs['desired_frequency']
            period = loop_grp.attrs['period']
            bin_step = np.ceil(100*period).astype(int)
            Z = np.sqrt(rl_coeff[pole]['R']**2 + (np.pi*2*rl_coeff[pole]['L']/period)**2)
            
            # get the data
            hp_data = filter_data(np.vstack([data_grp['hallprobe/data'][:]
                                 for data_grp in loop_grp.values()]), bin_step = bin_step)
            hp_data[:, 0] -= hp_data[0, 0]
            hx_data = filter_data(np.vstack([data_grp['hexapole/data'][:]
                                 for data_grp in loop_grp.values()]), bin_step = bin_step)
            hx_data[:, 0] -= hx_data[0, 0]
            
            # check if compliance reached
            voltage = np.max(np.abs(hx_data[:, pole+1])) * Z
            if voltage > 20:
                continue

            # create the desired signal
            t = hp_data[:, 0]
            # create the desired signal
            measured_signal = hp_data[:, pole+1].copy()
            desired_signal = damp[pole]*np.sin(2*np.pi/period*(t - 0.015))

            plt.close('all')
            fig = plt.figure(figsize=(20, 6), dpi=100)
            ax = fig.add_subplot()
            ax.plot(t, measured_signal, label='Measured signal')
            # make a note if compliance reached
#             if compliance_reached:
#                 ax.plot(t, desired_signal, label='Desired signal', color='red')
#                 # Create empty plot with blank marker containing the extra label
#                 ax.plot([], [], ' ', label="Compliance reached!")
#             else:
            ax.plot(t, desired_signal, label='Desired signal')
                
                
            # do the error analysis
            err = np.abs((measured_signal - desired_signal))
            idx = np.argmax(err)
            maxerr = err[idx]
            stderr = np.sqrt(np.mean(err**2))
            s = 'Maximum error: {:.2f}mT\nMSE error: {:.3f}mT'.format(maxerr, stderr)
#             print(s)
#             print(s)set_format(poles_dict[pole])+s)
                
            ax.set_xlabel('t [s]')
            ax.set_ylabel('B [mT]')
            plt.legend(loc='upper right')
            plt.title('Sin signal direction {} \nAmp: {:.2f}mT; Freq: {:.2f}Hz\n'.format(directions_dict[pole], damp[pole], 1/period)
                     +s)
            
            
            # add the error plot on the other axes
            ax2 = ax.twinx()
            ax2.plot(t, measured_signal - desired_signal, color=errcolor, alpha=0.8,  label='Error')
            ax2.set_ylabel('Error [V]', color=errcolor)
#             ax2.set_ylim([-20, 20])
            ax2.tick_params(axis='y', labelcolor=errcolor)

            plt.tight_layout()
            figname = 'sin_signal_pole{}_{}'.format(pole, i)
            print(figname)
            plt.savefig(os.path.join(out_folder, figname))
#             if i==10:
#                 break
#             break
#         break

#### Test the lag

In [84]:
fft = np.fft.rfft(measured_signal)
phase = np.angle(fft[np.argmax(np.absolute(fft))]) + np.pi/2
print('lag: ', phase / (np.pi*2/period))

lag:  -0.017811787769132562


### Test the random signal generation

In [ ]:
desired_hp[:, pole+1] - hp_data[np.logical_and(t>5, t<10), pole+1]

In [50]:
np.random.rand(3, 2)

array([[0.09426857, 0.1405991 ],
       [0.19047218, 0.30158876],
       [0.39420587, 0.44059002]])

In [52]:
np.float(3)

3.0

In [51]:
np.concatenate((np.random.rand(3, 2), np.random.rand(1, 2)), axis=0)

array([[0.30281928, 0.84447672],
       [0.46359257, 0.18444557],
       [0.44519537, 0.30148051],
       [0.07241305, 0.17995247]])

In [98]:
desired_hp

array([[ 0.        ,  4.19029792,  0.        ,  0.        ],
       [ 0.01      ,  2.05736912,  0.        ,  0.        ],
       [ 0.02      , -0.46886885,  0.        ,  0.        ],
       ...,
       [ 4.97      ,  4.51568967,  0.        ,  0.        ],
       [ 4.98      ,  5.52800981,  0.        ,  0.        ],
       [ 4.99      ,  5.43407599,  0.        ,  0.        ]])

In [97]:
desired_hp1 = np.roll(desired_hp[:, 1:], 1, axis=0)
desired_hp1

array([[5.43407599, 0.        , 0.        ],
       [4.19029792, 0.        , 0.        ],
       [2.05736912, 0.        , 0.        ],
       ...,
       [2.71992326, 0.        , 0.        ],
       [4.51568967, 0.        , 0.        ],
       [5.52800981, 0.        , 0.        ]])

In [103]:
# import the data
# pole = 0
# directions_dict = ['x', 'y', 'z']
# file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\magnet_calibration_test_hystcalib3.h5"
lag = 0.015
with h5py.File(file_path, 'r') as f:
    test_groups = f['test_response']
    for i, test_group in enumerate(test_groups.values()):
        # get the data
        hp_data = filter_data(test_group['hallprobe/data'][:])
        hp_data[:, 0] -= hp_data[0, 0]
        hx_data = filter_data(test_group['hallprobe/data'][:])
        hx_data[:, 0] -= hx_data[0, 0]
        desired_hp = filter_data(test_group['desired_output'][:])
        
        desired_hp[:, 1:] = np.roll(desired_hp[:, 1:], 1, axis=0)

        # show the signal
        t = hp_data[:, 0]
        td = desired_hp[:, 0]
        td -= td[0]

        plt.close('all')
        plt.figure(figsize=(8, 12), dpi=100)
        for pole in poles:
            ax = plt.subplot(3, 1, pole+1)
            ax.plot(td+5, desired_hp[:, pole+1], label='Desired signal')
            ax.plot(t, hp_data[:, pole+1], label='Measured signal')
            plt.xlabel('t [s]')
            plt.ylabel('B [mT]')
            plt.legend(loc='upper right')
            plt.xlim(5, 10)
            if np.max(np.abs(hp_data[:, pole+1]))<2:
                plt.ylim([-2, 2])
                
             # do the error analysis
            err = desired_hp[:, pole+1] - hp_data[t>=5, pole+1][:500]
            idx = np.argmax(np.abs(err))
            maxerr = np.abs(err[idx])
            stderr = np.sqrt(np.mean(err**2))
            s = 'Maximum error: {:.2f}mT\nStd error: {:.3f}mT'.format(maxerr, stderr)
            print(s)
            
            
            # add the error plot on the other axes
            ax2 = ax.twinx()
            ax2.plot(t[t>=5][:500], err, color=errcolor, alpha=0.6,  label='Error')
            ax2.set_ylabel('Error [mT]', color=errcolor)
#             ax2.set_ylim([-20, 20])
            ax2.tick_params(axis='y', labelcolor=errcolor)
    
            plt.title('Random signal direction {}, test {}\n'.format(directions_dict[pole], i)
                     +s)
            
#             break

        plt.tight_layout()
#         break
        plt.savefig(os.path.join(out_folder, 'random_signal_{}'.format(i)))

Maximum error: 3.76mT
Std error: 1.002mT
Maximum error: 0.48mT
Std error: 0.338mT
Maximum error: 0.80mT
Std error: 0.113mT
Maximum error: 2.44mT
Std error: 0.769mT
Maximum error: 3.90mT
Std error: 1.138mT
Maximum error: 4.19mT
Std error: 1.415mT
Maximum error: 3.92mT
Std error: 1.063mT
Maximum error: 3.05mT
Std error: 0.923mT
Maximum error: 2.21mT
Std error: 0.749mT
